# GAHN ALIOUNE BADARA BA 
# DIALLO AICHA

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import pandas as pd

In [210]:
pathdftraintext = '/xtraintext.txt' #Chemin vers le DataFrame Xtrain contenant les features Text
pathdftestext = '/xtesttext.txt' #Chemin vers le DataFrame Xtest contenant les features Text
pathdftrainnetwork = '/xtrainnetwork.txt' #Chemin vers le DataFrame Xtrain contenant les features Network Mining
pathdftestnetwork = '/xtestnetwork.txt' #Chemin vers le DataFrame Xtest contenant les features Network
#Nous avons modifié le fichier "labels_training.txt" pour faciliter son importation avec notre code. Nous avons juste enlevé la première ligne ('doc,class'). Nous le joignons dans notre dossier (vous pouvez le modifier dans votre fichier si vous préferez).
pathfile3= '/fake-news-prediction-toulouse/data_competition/labels_training.txt' # Chemin vers le fichier contenant les labels 

In [211]:
#Cette fonction prend en entrée un fichier de données numériques text et renvoie une liste de listes contenant chacune 
# une colonne du fichier
def readtxt(pathfile ,k ):
    data = [ int(z) for z in open(pathfile, 'r').read().split() ]
    liste = []
    j = 0
    while j <= (k-1):
        liste.append([data[i] for i in range(j, len(data),k)])
        j = j+1
    return(liste)

#### Lecture des dataframe Train et Text contenant les features Text et Network Mining

In [212]:
dfxtraintext = pd.read_csv(pathdftraintext) 
dfxtesttext = pd.read_csv(pathdftestext)
dfxtrainnetwork = pd.read_csv(pathdftrainnetwork)
dfxtestnetwork = pd.read_csv(pathdftestnetwork)

#### Combinaison des features Text Mining et Network Mining

In [213]:
dfxtrain = pd.merge(dfxtraintext, dfxtrainnetwork, on = 'doc')
dfxtest = pd.merge(dfxtesttext, dfxtestnetwork, on = 'doc' )

#### Importation des labels 

In [214]:
label = readtxt(pathfile3 , 2) 
label = pd.DataFrame({'doc':label[0],'class':label[1]})
label = label.set_index(label['doc'].values)

In [216]:
dfxtrain1 = dfxtrain.loc[:, 'classdoc':]

In [238]:
dfxtest1 = dfxtest.loc[:, 'classdoc':]

### Classifieur de Bayes

On utilise le classifieur de Bayes car c'est celui qui nous donne le meilleur score sans pour autant avoir besoin d'optimiser les paramètres

In [239]:
gnb = GaussianNB()
y_pred = gnb.fit(dfxtrain1, label['class'].values).predict(dfxtest1)
pred= pd.DataFrame(y_pred, index = dfxtest['doc'].values)
pred.insert(0, 'doc', pred.index)
pred.columns = ['doc', 'class']

### Logistic Regression

In [243]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(dfxtrain1, label['class'].values)
ylr = clf.predict(dfxtest1)
predlr= pd.DataFrame(ylr, index = dfxtest['doc'].values)
predlr.insert(0, 'doc', predlr.index)
predlr.columns = ['doc', 'class']

### Random Forest Classifier

In [246]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(dfxtrain1, label['class'].values)
ylrf = clf.predict(dfxtest1)
predrf= pd.DataFrame(ylrf, index = dfxtest['doc'].values)
predrf.insert(0, 'doc', predrf.index)
predrf.columns = ['doc', 'class']

#### Trouvons maintenant le meilleur modèle

In [261]:
predrf

,doc,class
2,2,0
4,4,0
7,7,0
16,16,0
17,17,0
18,18,0
24,24,1
34,34,0
40,40,1
43,43,0


Apres visualisation des résultats, nos 3 classifieurs nous prédise les memes resultats. 
On a une bonne précision et un bon rappel sur les fakes news, mais notre modèle classifie 2 ou 3 vrais news comme des fakes. 
Nous conservons quand meme le modèle Bayésien: 

In [262]:
pred.to_csv('/Submitbayes.txt', index=False, header=1 ) #Save predictions